In [1]:
# For data manipulation

import pandas as pd

import numpy as np 

# machine learning classification

from sklearn.svm import SVC

from sklearn.metrics import scorer

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# To plot
%matplotlib inline
import matplotlib.pyplot as plt

import seaborn

# To fetch data

from pandas_datareader import data as pdr


In [2]:
import glob
import pandas as pd
pattern='*.csv'
hong = glob.glob(pattern)
len(hong)

62

In [3]:
hkong = pd.read_csv(hong[0])
hkong = hkong.drop('Adj Close', axis=1)
hkong.columns = ['Date', 'Open' + '_'+str(0), 'High'+'_'+str(0), 'Low' +'_'+str(0), 'Close'+ '_'+str(0), 'Volume'+ '_'+ str(0)]

for date in hkong['Date']:
    hkong['Date'] = pd.to_datetime(hkong['Date'])
hkong =hkong.sort_values(by='Date')

hkong['Volume'+ '_'+ str(0)] = hkong['Volume'+ '_'+ str(0)].fillna(0)
for col in hkong:
    hkong[col] = hkong[col].fillna(method='ffill')
    
#hkong['Target'] = np.where(hkong['Close'].shift(-1) > hkong['Close'],int(1),int(-1))
hkong.shape

(1234, 6)

In [4]:
hkong['Open' + '_'+str(0)] = ((hkong['Open' + '_'+str(0)] - hkong['Open' + '_'+str(0)].shift(1))/hkong['Open' + '_'+str(0)].shift(1)) *100

hkong['High' + '_'+str(0)] = ((hkong['High' + '_'+str(0)] - hkong['High' + '_'+str(0)].shift(1))/hkong['High' + '_'+str(0)].shift(1)) *100

hkong['Low' + '_'+str(0)] = ((hkong['Low' + '_'+str(0)] - hkong['Low' + '_'+str(0)].shift(1))/hkong['Low' + '_'+str(0)].shift(1)) *100

hkong['Close' + '_'+str(0)] = ((hkong['Close' + '_'+str(0)] - hkong['Close' + '_'+str(0)].shift(1))/hkong['Close' + '_'+str(0)].shift(1)) *100

hkong = hkong.drop(0, axis=0)


print hkong.shape
hkong.head()

#hkong['Open' + '_'+str(0)] = hkong['Open' + '_'+str(0)].drop(hkong['Open' + '_'+str(0)].index[0])


(1233, 6)


,Date,Open_0,High_0,Low_0,Close_0,Volume_0
1,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412
2,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548
3,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634
4,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274
5,2013-03-18,-2.843602,-2.351097,-1.620746,0.646200,2427267


In [5]:
#run only once
for i in range(1,len(hong)):
    df = pd.read_csv(hong[i]) 
    
    if 'Adj Close' in df:
        df = df.drop('Adj Close', axis=1) 
    
    df.columns = ['Date', 'Open' + '_'+str(i), 'High'+'_'+str(i), 'Low' +'_'+str(i), 'Close'+'_'+str(i), 'Volume'+'_'+str(i)]
    
    #formatting date
    for date in df['Date']:
        df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date')
    
    #imputing missing values
    df['Volume'+ '_'+ str(i)] = df['Volume'+ '_'+ str(i)].fillna(0)
    for col in df:
        df[col] = df[col].fillna(method='ffill')
    df['Open' + '_'+str(i)] = ((df['Open' + '_'+str(i)] - df['Open' + '_'+str(i)].shift(1))/df['Open' + '_'+str(i)].shift(1)) *100

    df['High' + '_'+str(i)] = ((df['High' + '_'+str(i)] - df['High' + '_'+str(i)].shift(1))/df['High' + '_'+str(i)].shift(1)) *100

    df['Low' + '_'+str(i)] = ((df['Low' + '_'+str(i)] - df['Low' + '_'+str(i)].shift(1))/df['Low' + '_'+str(i)].shift(1)) *100

    df['Close' + '_'+str(i)] = ((df['Close' + '_'+str(i)] - df['Close' + '_'+str(i)].shift(1))/df['Close' + '_'+str(i)].shift(1)) *100

    df = df.drop(0, axis=0)
    
    hkong = pd.merge(hkong, df, how='outer', on='Date')

hkong.shape


(1555, 311)

In [6]:
#for i in range(len(hong)):
   # print hkong.loc[:, 'Volume'+'_'+str(i)].head()

In [7]:
#hkong.to_csv('E:/hkongpercent-1_missing.csv')  ---->not needed

In [8]:

for date in hkong['Date']:
    hkong['Date'] = pd.to_datetime(hkong['Date'])
hkong =hkong.sort_values(by='Date')

In [9]:
hkong.head()

,Date,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,...,Open_60,High_60,Low_60,Close_60,Volume_60,Open_61,High_61,Low_61,Close_61,Volume_61
1233,2013-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,...,-0.238700,0.531070,-1.382659,-1.038177,117600.0,0.213418,-0.321168,-0.190709,-0.547478,2738000.0
1,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,...,-1.168739,-2.012475,-0.511300,-0.989981,91000.0,-0.250032,-0.194338,-0.302631,0.010004,2224400.0
2,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,...,-1.156576,-0.442113,0.002709,0.278674,80200.0,-0.205514,-0.557313,-0.202575,-0.547182,2150200.0
3,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,...,0.600239,1.575284,-0.146681,0.357832,116400.0,-0.159327,-0.034864,-0.268680,-0.305210,2530400.0


In [10]:
for i in range(0,len(hong)):
    hkong['Volume'+ '_'+ str(i)] = hkong['Volume'+ '_'+ str(i)].fillna(0)
for col in hkong.columns:
    hkong[col] = hkong[col].fillna(method='ffill')
hkong.head()

,Date,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,...,Open_60,High_60,Low_60,Close_60,Volume_60,Open_61,High_61,Low_61,Close_61,Volume_61
1233,2013-03-11,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
0,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,...,-0.238700,0.531070,-1.382659,-1.038177,117600.0,0.213418,-0.321168,-0.190709,-0.547478,2738000.0
1,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,...,-1.168739,-2.012475,-0.511300,-0.989981,91000.0,-0.250032,-0.194338,-0.302631,0.010004,2224400.0
2,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,...,-1.156576,-0.442113,0.002709,0.278674,80200.0,-0.205514,-0.557313,-0.202575,-0.547182,2150200.0
3,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,...,0.600239,1.575284,-0.146681,0.357832,116400.0,-0.159327,-0.034864,-0.268680,-0.305210,2530400.0


In [11]:

hkong = hkong.fillna(0)

In [12]:
hkong.head()

,Date,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,...,Open_60,High_60,Low_60,Close_60,Volume_60,Open_61,High_61,Low_61,Close_61,Volume_61
1233,2013-03-11,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
0,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,...,-0.238700,0.531070,-1.382659,-1.038177,117600.0,0.213418,-0.321168,-0.190709,-0.547478,2738000.0
1,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,...,-1.168739,-2.012475,-0.511300,-0.989981,91000.0,-0.250032,-0.194338,-0.302631,0.010004,2224400.0
2,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,...,-1.156576,-0.442113,0.002709,0.278674,80200.0,-0.205514,-0.557313,-0.202575,-0.547182,2150200.0
3,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,...,0.600239,1.575284,-0.146681,0.357832,116400.0,-0.159327,-0.034864,-0.268680,-0.305210,2530400.0


In [13]:
hkong.isnull().sum()[hkong.isnull().sum() !=0]

Series([], dtype: int64)

In [14]:
for i in range(30):
    hkong['Target'+'_'+str(i)] = np.where(hkong['Close'+'_'+str(i)]>0,1,0)
    
hkong.shape

(1555, 341)

In [15]:
hkong.head()

,Date,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,...,Target_20,Target_21,Target_22,Target_23,Target_24,Target_25,Target_26,Target_27,Target_28,Target_29
1233,2013-03-11,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
0,2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,...,0,0,0,1,0,0,0,0,0,0
1,2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,...,0,0,0,0,0,0,0,0,0,0
2,2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,...,0,1,0,0,0,0,1,1,0,0
3,2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,...,0,0,0,0,0,0,0,0,0,0


In [16]:
hkong.index = hkong.Date
hkong = hkong.drop('Date', axis=1)
hkong.shape

(1555, 340)

In [17]:
hkong.head()

,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,Volume_1,...,Target_20,Target_21,Target_22,Target_23,Target_24,Target_25,Target_26,Target_27,Target_28,Target_29
Date,,,,,,,,,,,,,,,,,,,,,
2013-03-11,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,23618156.0,...,0,0,0,1,0,0,0,0,0,0
2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,31644955.0,...,0,0,0,0,0,0,0,0,0,0
2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,21286152.0,...,0,1,0,0,0,0,1,1,0,0
2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,44707880.0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
hkong1 = hkong.copy()
hkong1.head()

,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,Volume_1,...,Target_20,Target_21,Target_22,Target_23,Target_24,Target_25,Target_26,Target_27,Target_28,Target_29
Date,,,,,,,,,,,,,,,,,,,,,
2013-03-11,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
2013-03-12,-0.776397,-0.613485,0.315451,0.469474,1597412.0,1.754380,0.143882,0.441182,-0.436673,23618156.0,...,0,0,0,1,0,0,0,0,0,0
2013-03-13,-0.469490,-0.925938,-2.830186,-3.115259,2922548.0,-0.574716,-0.574716,-1.903373,-0.877190,31644955.0,...,0,0,0,0,0,0,0,0,0,0
2013-03-14,-3.930818,-2.336443,-1.294498,-0.321543,2704634.0,-1.589593,-1.589593,0.149251,0.294979,21286152.0,...,0,1,0,0,0,0,1,1,0,0
2013-03-15,3.600658,1.754386,1.147541,-0.161287,3487274.0,1.321589,2.349483,1.639341,1.470588,44707880.0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
#feeding next day's target
for i in range(30):
    hkong1['Target'+'_'+str(i)] = hkong1['Target'+'_'+str(i)].shift(-1)

In [20]:
#remove NAN row (only one)
hkong1 = hkong1.iloc[:1554, :]
hkong1.tail()

,Open_0,High_0,Low_0,Close_0,Volume_0,Open_1,High_1,Low_1,Close_1,Volume_1,...,Target_20,Target_21,Target_22,Target_23,Target_24,Target_25,Target_26,Target_27,Target_28,Target_29
Date,,,,,,,,,,,,,,,,,,,,,
2018-03-02,-2.580645,-1.963262,-2.181099,-1.107490,4187815.0,0.466207,-1.523229,-0.623050,-1.829262,31294395.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-03-04,-2.580645,-1.963262,-2.181099,-1.107490,0.0,0.466207,-1.523229,-0.623050,-1.829262,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-03-05,-1.125826,-0.775192,-1.081085,-2.042165,4760318.0,-0.850741,-0.464043,-2.507834,-2.872675,25358211.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0
2018-03-06,3.148022,1.302083,2.459021,2.891730,5038858.0,-1.560062,0.621604,1.446940,3.197448,23416389.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-03-07,-2.597403,-1.156814,-1.799998,-2.614379,5324814.0,0.871639,-1.389964,-0.633910,-2.711083,22738854.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0


In [21]:
target=hkong1.iloc[:, 310:339].columns
target

Index([u'Target_0', u'Target_1', u'Target_2', u'Target_3', u'Target_4',
       u'Target_5', u'Target_6', u'Target_7', u'Target_8', u'Target_9',
       u'Target_10', u'Target_11', u'Target_12', u'Target_13', u'Target_14',
       u'Target_15', u'Target_16', u'Target_17', u'Target_18', u'Target_19',
       u'Target_20', u'Target_21', u'Target_22', u'Target_23', u'Target_24',
       u'Target_25', u'Target_26', u'Target_27', u'Target_28'],
      dtype='object')

In [22]:
train = hkong1.iloc[:1000, :]
test = hkong1.iloc[1000:, :]
#print train.tail()
#print test.head()

In [23]:
Xtrain = train.iloc[:, :310]
Xtest = test.iloc[:, :310]
sc=StandardScaler()
Xtrain = sc.fit_transform(Xtrain)
Xtest = sc.fit_transform(Xtest)
print Xtrain.shape
print Xtest.shape

(1000, 310)
(554, 310)


In [24]:
scores = []
test_acc =[]
for col in target:
    ytrain = train[col]
    ytest = test[col]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    classifier = LogisticRegression()
    classifier.fit(Xtrain, ytrain)

    cv_score = np.mean(cross_val_score(classifier, Xtest, ytest, cv=10, scoring='roc_auc'))
    y_pred_prob = classifier.predict_proba(Xtest)[:, 1]
    #roc_score = roc_auc_score(ytest, y_pred_prob)
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(col, cv_score))

    
    accuracy_train = accuracy_score(ytrain, classifier.predict(Xtrain))

    accuracy_test = accuracy_score(ytest, classifier.predict(Xtest))
    test_acc.append(accuracy_test*100)
    #print('\nTrain Accuracy:{: .2f}%'.format(accuracy_train*100))

    #print('Test Accuracy:{: .2f}%'.format(accuracy_test*100))
    

print('Total ROC-AUC score is {}'.format(np.mean(scores)))
print ('Mean Test Accuracy is {: .2f}%'.format(np.mean(test_acc)))
test_acc = map(lambda x:round(x,2), test_acc)
test_acc_LR_1 =test_acc
print test_acc_LR_1

CV score for class Target_0 is 0.548791278964
CV score for class Target_1 is 0.577250221043
CV score for class Target_2 is 0.497067141033
CV score for class Target_3 is 0.516053267163
CV score for class Target_4 is 0.550631201493
CV score for class Target_5 is 0.463642473118
CV score for class Target_6 is 0.571191335102
CV score for class Target_7 is 0.542408740685
CV score for class Target_8 is 0.516999104168
CV score for class Target_9 is 0.550520071685
CV score for class Target_10 is 0.581364901595
CV score for class Target_11 is 0.540350039193
CV score for class Target_12 is 0.524369175627
CV score for class Target_13 is 0.49741821397
CV score for class Target_14 is 0.526041054282
CV score for class Target_15 is 0.529069653208
CV score for class Target_16 is 0.522409231892
CV score for class Target_17 is 0.560372194855
CV score for class Target_18 is 0.487110918972
CV score for class Target_19 is 0.47204598842
CV score for class Target_20 is 0.571798029557
CV score for class Target

In [25]:
scores = []
test_acc =[]
for col in target:
    ytrain = train[col]
    ytest = test[col]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    classifier = SVC()
    classifier.fit(Xtrain, ytrain)

    cv_score = np.mean(cross_val_score(classifier, Xtest, ytest, cv=10, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(col, cv_score))

    
    accuracy_train = accuracy_score(ytrain, classifier.predict(Xtrain))

    accuracy_test = accuracy_score(ytest, classifier.predict(Xtest))
    test_acc.append(accuracy_test*100)
    #print('\nTrain Accuracy:{: .2f}%'.format(accuracy_train*100))

    #print('Test Accuracy:{: .2f}%'.format(accuracy_test*100))
    #submission[class_name] = classifier.predict_proba(Xtest)[:, 1]


print('Total CV score is {}'.format(np.mean(scores)))

print ('Mean Test Accuracy is {: .2f}%'.format(np.mean(test_acc)))
test_acc = map(lambda x:round(x,2), test_acc)
test_acc_SVC_1 =test_acc
print test_acc_SVC_1

CV score for class Target_0 is 0.550018244846
CV score for class Target_1 is 0.560442086649
CV score for class Target_2 is 0.550802773217
CV score for class Target_3 is 0.521377667494
CV score for class Target_4 is 0.518904291749
CV score for class Target_5 is 0.506611223118
CV score for class Target_6 is 0.638837665782
CV score for class Target_7 is 0.560520960521
CV score for class Target_8 is 0.54376994625
CV score for class Target_9 is 0.609119354839
CV score for class Target_10 is 0.571265350231
CV score for class Target_11 is 0.478176877747
CV score for class Target_12 is 0.597176523297
CV score for class Target_13 is 0.489470380195
CV score for class Target_14 is 0.592667303547
CV score for class Target_15 is 0.558296492779
CV score for class Target_16 is 0.588697318008
CV score for class Target_17 is 0.638724685276
CV score for class Target_18 is 0.522379364295
CV score for class Target_19 is 0.466975682382
CV score for class Target_20 is 0.572760445174
CV score for class Targe

In [26]:
scores = []
test_acc =[]
for col in target:
    ytrain = train[col]
    ytest = test[col]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    classifier = RandomForestClassifier()
    classifier.fit(Xtrain, ytrain)

    cv_score = np.mean(cross_val_score(classifier, Xtest, ytest, cv=10, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(col, cv_score))

    
    accuracy_train = accuracy_score(ytrain, classifier.predict(Xtrain))

    accuracy_test = accuracy_score(ytest, classifier.predict(Xtest))
    test_acc.append(accuracy_test*100)
    #print('\nTrain Accuracy:{: .2f}%'.format(accuracy_train*100))

    #print('Test Accuracy:{: .2f}%'.format(accuracy_test*100))
    #submission[class_name] = classifier.predict_proba(Xtest)[:, 1]



print('Total CV score is {}'.format(np.mean(scores)))

print ('Mean Test Accuracy is {: .2f}%'.format(np.mean(test_acc)))
test_acc = map(lambda x:round(x,2), test_acc)
test_acc_RFC_1 =test_acc
print test_acc_RFC_1

CV score for class Target_0 is 0.513872924649
CV score for class Target_1 is 0.524247126437
CV score for class Target_2 is 0.517087789286
CV score for class Target_3 is 0.512105707196
CV score for class Target_4 is 0.504615945995
CV score for class Target_5 is 0.468208165323
CV score for class Target_6 is 0.603616445623
CV score for class Target_7 is 0.523064906039
CV score for class Target_8 is 0.497485285266
CV score for class Target_9 is 0.586240143369
CV score for class Target_10 is 0.526108327603
CV score for class Target_11 is 0.475378103916
CV score for class Target_12 is 0.533466845878
CV score for class Target_13 is 0.510944739169
CV score for class Target_14 is 0.529865572772
CV score for class Target_15 is 0.518113272424
CV score for class Target_16 is 0.549831235176
CV score for class Target_17 is 0.578186006203
CV score for class Target_18 is 0.47877140975
CV score for class Target_19 is 0.471441521919
CV score for class Target_20 is 0.52077631819
CV score for class Target

In [27]:
scores = []
test_acc =[]
for col in target:
    ytrain = train[col]
    ytest = test[col]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    classifier = GradientBoostingClassifier()
    classifier.fit(Xtrain, ytrain)

    cv_score = np.mean(cross_val_score(classifier, Xtest, ytest, cv=10, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(col, cv_score))

    
    accuracy_train = accuracy_score(ytrain, classifier.predict(Xtrain))

    accuracy_test = accuracy_score(ytest, classifier.predict(Xtest))
    test_acc.append(accuracy_test*100)
    #print('\nTrain Accuracy:{: .2f}%'.format(accuracy_train*100))

    print('Test Accuracy:{: .2f}%'.format(accuracy_test*100))
    #submission[class_name] = classifier.predict_proba(Xtest)[:, 1]



print('Total CV score is {}'.format(np.mean(scores)))

print ('Mean Test Accuracy is {: .2f}%'.format(np.mean(test_acc)))
test_acc = map(lambda x:round(x,2), test_acc)
test_acc_GBC_1 =test_acc
print test_acc_GBC_1

CV score for class Target_0 is 0.592870826492
Test Accuracy: 58.84%
CV score for class Target_1 is 0.590572060124
Test Accuracy: 54.87%
CV score for class Target_2 is 0.484729239471
Test Accuracy: 53.43%
CV score for class Target_3 is 0.519898097601
Test Accuracy: 60.65%
CV score for class Target_4 is 0.554556825678
Test Accuracy: 59.03%
CV score for class Target_5 is 0.461176915323
Test Accuracy: 52.71%
CV score for class Target_6 is 0.610929619805
Test Accuracy: 64.26%
CV score for class Target_7 is 0.544826358188
Test Accuracy: 60.83%
CV score for class Target_8 is 0.556917149576
Test Accuracy: 58.12%
CV score for class Target_9 is 0.669524014337
Test Accuracy: 63.54%
CV score for class Target_10 is 0.499226184628
Test Accuracy: 57.40%
CV score for class Target_11 is 0.485824952409
Test Accuracy: 58.48%
CV score for class Target_12 is 0.595869175627
Test Accuracy: 65.52%
CV score for class Target_13 is 0.566842617153
Test Accuracy: 56.50%
CV score for class Target_14 is 0.4957807557

In [30]:
testacc_df = pd.DataFrame([hong[:29], test_acc_LR_1, test_acc_SVC_1, test_acc_RFC_1, test_acc_GBC_1]).T
testacc_df.columns = ['Stock Components', 'Logistic Regression % (0.0) (1-)', 'SVM % (0.0) (1-)', 'Random Forest % (0.0) (1-)', 'Gradient Boosting % (0.0) (1-)']
testacc_df['Stock Components'] = map(lambda x:x.replace('.csv', ''), testacc_df['Stock Components'])
testacc_df

,Stock Components,Logistic Regression % (0.0) (1-),SVM % (0.0) (1-),Random Forest % (0.0) (1-),Gradient Boosting % (0.0) (1-)
0,AAC Technologies Holdings Inc. (2018.HK),51.08,53.43,49.1,58.84
1,AIA Group Limited (1299.HK),54.15,54.87,47.83,54.87
2,BOC Hong Kong (Holdings) Limited (2388.HK),53.25,55.78,55.05,53.43
3,China Life Insurance Company Limited (2628.HK),56.5,56.5,60.29,60.65
4,China Mengniu Dairy Company Limited (2319.HK),53.07,55.96,53.97,59.03
5,China Mobile Limited (0941.HK),54.33,54.33,51.62,52.71
6,China Petroleum & Chemical Corporation(0386.HK),55.23,61.01,58.84,64.26
7,China Resources Land Limited (1109.HK),53.79,55.96,52.53,60.83
8,China Resources Power Holdings Company Limited...,53.07,54.33,50,58.12
9,CITIC Limited (0267.HK),55.6,55.78,55.96,63.54


In [31]:
testacc_df.to_csv('E:/hangseng1/span1_0p0.csv', index=False)